# DiscreteLab №2: Huffman, LZW and LZ77
## Held by: Eugene Bevz; Completeness: 80%

In [14]:
from typing import List, Tuple

## LZ77 - Old But Fresh
Here we attempted to code LZ77 compression and decompression algorithm, which is used to compress messages or files.

In [15]:
def read_file(filename: str) -> str:
    """
    Read file and return a solid string with data.

    :param filename: path to file
    :return: string with text
    """
    with open(filename, 'r') as file:
        return file.read()


print(read_file('short'))
print(read_file('medium'))
print(read_file('long'))

The steadfast love of the Lord never ceases; his mercies never come to an end; they are new every morning; great is your faithfulness.

So we do not lose heart. Though our outer self is wasting away, our inner self is being renewed day by day. For this light momentary affliction is preparing for us an eternal weight of glory beyond all comparison, as we look not to the things that are seen but to the things that are unseen. For the things that are seen are transient, but the things that are unseen are eternal.



1 The words of king Lemuel, the prophecy that his amother taught him.

2 What, my son? and what, the son of my womb? and what, the son of my vows?

3 Give not thy strength unto awomen, nor thy ways to that which destroyeth kings.

4 It is not for kings, O Lemuel, it is not for kings to drink awine; nor for princes strong drink:

5 Lest they drink, and forget the law, and pervert the judgment of any of the afflicted.

6 Give strong drink unto him that is ready to perish, and wi

## Initialise class and compress/decompress methods
The `LZ77` can't be described as super unique class, as it only has two methods: `compress` and `decompress`, the names of which tell us what they are supposed to do.


`compress` generates a list of codes in format __<offset, length, next>__, that represent some part of data that repeats itself. By doing so, we receive a list, each element of which tells us: "step back _offset_ steps, repeat that part _length_ times and put _next_ after it".


`decompress` uses the mentioned list of codes and rebuilds the initial string with it. Decompressing is actually similar to compressing, as it looks at every __<offset, length, next>__ and does almost the same as compressing: step back, copy, add symbol in the end.


The list of codes may vary, depending on searching window and buffer size. The bigger it is, the more accurate compression is, however memory consumption rises accordingly.
Currently, methods are not perfectly accurate, perhaps they're able to handle tha major part of work.

In [16]:
class LZ77:
    def __init__(self, buffer_size: int = 5, box_size: int = 5):
        self.buffer_size = buffer_size
        self.box_size = box_size

    def compress(self, message: str) -> List[Tuple[int, int, str]]:
        """
        Compress message.

        :param message: a message to be compressed
        :return: list of tuples(<offset, length, next_character>)
        """
        cursor = 0
        compressed = []
        while cursor < len(message):
            pattern = (0, 0, '')
            # make a "window" to look over text parts =====================
            box_begin = max(0, cursor - self.box_size)
            box_end = cursor + self.buffer_size
            # make a buffer to store data =================================
            buffer_end = min(box_end, len(message))
            buffer_begin = max(0, buffer_end - self.buffer_size)

            buffer = message[buffer_begin:buffer_end]
            box = message[box_begin:cursor]

            for i in range(len(buffer)):
                j = box.rfind(buffer[:i + 1])
                if j != -1:
                    pattern = (cursor - box_begin - j, i + 1, buffer[i + 1])
            if pattern == (0, 0, ''):
                # no pattern found, encode the current character
                pattern = (0, 0, message[cursor])
                cursor += 1
            else:
                cursor += pattern[1]
            compressed.append(pattern)

        return compressed

    def decompress(self, compressed: List[Tuple[int, int, str]]) -> str:
        """
        Decompress data.

        :param compressed: list of tuples(<offset, length, next_character>)
        :return: original string
        """
        decompressed = ''
        last_char = ''
        for pattern in compressed:
            if pattern[0] == pattern[1] == 0:
                if pattern[2] != last_char:
                    decompressed += pattern[2]
                    last_char = pattern[2]
            else:
                offset = len(decompressed) - pattern[0]
                for i in range(pattern[1]):
                    decompressed += decompressed[offset + i]
                    last_char = decompressed[-1]
                decompressed += pattern[2]
                last_char = pattern[2]
        return decompressed

In [17]:
short_to_compress = read_file('short')
medium_to_compress = read_file('medium')
long_to_compress = read_file('long')

lz77 = LZ77()

compressed_short = lz77.compress(short_to_compress)
print("Compressed 1: ", compressed_short, '\n')
compressed_medium = lz77.compress(medium_to_compress)
print("Compressed 2: ", compressed_medium, '\n')
# compressed_long = lz77.compress(long_to_compress)
# print("Compressed 3: ", compressed_long, '\n')

print("=============================================================")

decompressed_short = lz77.decompress(compressed_short)
print("Decompressed 1: ", decompressed_short, '\n')
decompressed_medium = lz77.decompress(compressed_medium)
print("Decompressed 2: ", decompressed_medium, '\n')
# decompressed_long = lz77.decompress(compressed_long)
# print("Decompressed 3: ", decompressed_long, '\n')


Compressed 1:  [(0, 0, 'T'), (0, 0, 'h'), (0, 0, 'e'), (0, 0, ' '), (0, 0, 's'), (0, 0, 't'), (4, 1, 'a'), (0, 0, 'a'), (0, 0, 'd'), (0, 0, 'f'), (3, 1, 's'), (0, 0, 's'), (0, 0, 't'), (0, 0, ' '), (0, 0, 'l'), (0, 0, 'o'), (0, 0, 'v'), (0, 0, 'e'), (5, 1, 'o'), (4, 1, 'f'), (0, 0, 'f'), (3, 1, 't'), (0, 0, 't'), (0, 0, 'h'), (0, 0, 'e'), (4, 1, 'L'), (0, 0, 'L'), (0, 0, 'o'), (0, 0, 'r'), (0, 0, 'd'), (5, 1, 'n'), (0, 0, 'n'), (0, 0, 'e'), (0, 0, 'v'), (2, 1, 'r'), (0, 0, 'r'), (0, 0, ' '), (0, 0, 'c'), (4, 1, 'a'), (0, 0, 'a'), (0, 0, 's'), (3, 1, 's'), (2, 1, ';'), (0, 0, ';'), (0, 0, ' '), (0, 0, 'h'), (0, 0, 'i'), (5, 1, ' '), (4, 1, 'm'), (0, 0, 'm'), (0, 0, 'e'), (0, 0, 'r'), (0, 0, 'c'), (0, 0, 'i'), (4, 1, 's'), (0, 0, 's'), (0, 0, ' '), (0, 0, 'n'), (4, 1, 'v'), (0, 0, 'v'), (2, 1, 'r'), (0, 0, 'r'), (0, 0, ' '), (0, 0, 'c'), (0, 0, 'o'), (0, 0, 'm'), (0, 0, 'e'), (5, 1, 't'), (0, 0, 't'), (5, 1, ' '), (3, 1, 'a'), (0, 0, 'a'), (0, 0, 'n'), (3, 1, 'e'), (0, 0, 'e'), (3, 1, 'd